OPTIMIZACIÓN EN INGENIERÍA

Trabajo de investigación: Job Shop Scheduling

Nombres: David Morales P. y Claudio Muñoz M.
         

In [113]:
import Pkg # Importa el administrador de paquetes Pkg
Pkg.add("JuMP") # Instala el paquete JuMP

   Resolving package versions...
   Installed MUMPS_seq_jll ── v500.500.101+0
   Installed METIS_jll ────── v5.1.2+0
   Installed ASL_jll ──────── v0.1.3+0
   Installed OpenBLAS32_jll ─ v0.3.17+0
   Installed Ipopt_jll ────── v300.1400.1000+0
   Installed Ipopt ────────── v1.2.1
    Updating `C:\Users\claud\.julia\environments\v1.8\Project.toml`
  [b6b21f68] + Ipopt v1.2.1
    Updating `C:\Users\claud\.julia\environments\v1.8\Manifest.toml`
  [b6b21f68] + Ipopt v1.2.1
  [ae81ac8f] + ASL_jll v0.1.3+0
⌅ [9cc047cb] + Ipopt_jll v300.1400.1000+0
  [d00139f3] + METIS_jll v5.1.2+0
  [d7ed1dd3] + MUMPS_seq_jll v500.500.101+0
⌅ [656ef2d0] + OpenBLAS32_jll v0.3.17+0
        Info Packages marked with ⌅ have new versions available but compatibility constraints restrict them from upgrading. To see why use `status --outdated -m`
Precompiling project...
  ✓ ASL_jll
  ✓ METIS_jll
  ✓ OpenBLAS32_jll
  ✓ MUMPS_seq_jll
  ✓ Ipopt_jll
  ✓ Ipopt
  6 dependencies successfully precompiled in 17 seconds. 52 al

Es necesario instalar un Solver (paquete con métodos de optimización implementados, por ejemplo Simplex). Se instalará el Solver de IBM CPLEX escribiendo el siguiente comando.

In [82]:
ENV["CPLEX_STUDIO_BINARIES"] = "C:\\Program Files\\IBM\\ILOG\\CPLEX_Studio2211\\cplex\\bin\\x64_win64\\"
import Pkg
Pkg.add("CPLEX")
Pkg.build("CPLEX")

    Updating registry at `C:\Users\claud\.julia\registries\General.toml`
   Installed GLPK ─────────────── v1.1.2
   Installed MutableArithmetics ─ v1.3.0
   Installed JuMP ─────────────── v1.11.1
   Installed MathOptInterface ─── v1.16.0
    Updating `C:\Users\claud\.julia\environments\v1.8\Project.toml`
  [a076750e] ↑ CPLEX v0.6.0 ⇒ v1.0.0
  [60bf3e95] ↑ GLPK v0.14.14 ⇒ v1.1.2
  [2e9cd046] ↑ Gurobi v0.9.14 ⇒ v1.0.1
  [b6b21f68] ↑ Ipopt v0.7.0 ⇒ v1.2.1
  [4076af6c] ↑ JuMP v0.21.10 ⇒ v1.11.1
  [67920dd8] ↑ KNITRO v0.10.0 ⇒ v0.13.2
    Updating `C:\Users\claud\.julia\environments\v1.8\Manifest.toml`
  [b99e7846] - BinaryProvider v0.5.10
  [a076750e] ↑ CPLEX v0.6.0 ⇒ v1.0.0
  [49dc2e85] - Calculus v0.5.1
  [60bf3e95] ↑ GLPK v0.14.14 ⇒ v1.1.2
  [2e9cd046] ↑ Gurobi v0.9.14 ⇒ v1.0.1
  [cd3eb016] - HTTP v0.9.17
  [83e8ac13] - IniFile v0.5.1
  [b6b21f68] ↑ Ipopt v0.7.0 ⇒ v1.2.1
  [92d709cd] ↑ IrrationalConstants v0.1.1 ⇒ v0.2.2
  [7d188eb4] - JSONSchema v0.3.4
  [4076af6c] ↑ JuMP v0.21.10 ⇒ v

Status `C:\Users\claud\.julia\environments\v1.8\Project.toml`
  [a076750e] CPLEX v1.0.0
  [60bf3e95] GLPK v1.1.2
  [2e9cd046] Gurobi v1.0.1
  [7073ff75] IJulia v1.24.0
  [b6b21f68] Ipopt v1.2.1
  [4076af6c] JuMP v1.11.1
  [67920dd8] KNITRO v0.13.2


Se definen las funciones que se utilizaran

In [1]:
using JuMP
using CPLEX


# Funcion para modelar y resolver el problema JSSP
function solve_job_shop_scheduling(J, M, O, M_i, p_i, T, N_j)
    # Se define un solver
    model = Model(CPLEX.Optimizer)

    # variables de decision
    @variable(model, x[i=1:length(O), j=1:length(O)], Bin)  # Precedencia de las operaciones
    @variable(model, t[i=1:length(O)])  # tiempo de inicio de las operaciones
    @variable(model, Cmax >= 0) # Makespan con limite superior T
    
    # Funcion objetivo
    @objective(model, Min, Cmax)

    # restricciones
    for i in O
        @constraint(model, t[i] >= 0)
    end
    
    for j in J, i in O
        if j > 1
            if (N_j[j-1] + 1) <= i <= (N_j[j]-1)
                @constraint(model, t[i+1] >= t[i] + p_i[i])
            end
        end
    end
    
    for j in J
        @constraint(model, Cmax >= t[N_j[j]] + p_i[N_j[j]])
    end
    
    for i in O, j in O
        if i != j && M_i[i] == M_i[j]
            @constraint(model, t[j] >= t[i] + p_i[i] * x[i, j] - T * (1 - x[i, j]))
            @constraint(model, t[i] >= t[j] + p_i[j] * (1 - x[i, j]) - T * x[i, j])
        end
    end

    # Se resuleve el modelo
    optimize!(model)

    # Se obtienen las soluciones
    makespan = objective_value(model)
    start_times = value.(t)

    return makespan, start_times
end

# Funcion para trasnformar matriz NxN a vector
function matrix_to_vector(O, J, Machine, time)
    op_num_job = length(O)/length(J)
    M_i= zeros(Int64,length(O))
    p_i = zeros(Int64,length(O))
    contador = 1
    for j in J
        for i in 1:Int(op_num_job)
            if contador <= length(O)
                M_i[contador] = Machine[j, i]
                p_i[contador] = time[j, i]
                contador = contador + 1
            end
        end
    end
    return M_i, p_i
end

# Funcion para calcular el numero de operaciones en la j-esima tarea
function calculate_N_J(O, J)
    op_num_job = length(O)/length(J)
    N_j=zeros(Int64,length(J))
    for j in 1:length(J)
        for i in 1:op_num_job
            if j > 1
                N_j[j] = N_j[j - 1] + i
            else
                N_j[j]= N_j[j] + 1
            end
        end
    end
    return N_j
end

calculate_N_J (generic function with 1 method)

CASO 1: 3 Jobs, 3 machines, 6 operations, T=1000

In [3]:
using JuMP
using GLPK

# Ejemplo de uso
J = 1:3  # conjunto de tareas
M = 1:2  # conjunto de maquinas
O = 1:6  # conjunto de operaciones

M_i = [1, 2, 1, 2, 1, 2]  # Maquina asignada para cada operacion

p_i = [2, 3, 1, 2, 2, 1]  # tiempo de procesamiento para cada operacion

T = 1000    # Upper bound para makespan

N_j = [2, 4, 6]  # Numero total de operaciones para las primeras j tareas

makespan, start_times = solve_job_shop_scheduling(J, M, O, M_i, p_i, T, N_j)

println("Makespan: ", makespan)
println("Start times: ", start_times)

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
Tried aggregator 1 time.
MIP Presolve eliminated 6 rows and 24 columns.
Reduced MIP has 29 rows, 19 columns, and 82 nonzeros.
Reduced MIP has 12 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 29 rows, 19 columns, and 82 nonzeros.
Reduced MIP has 12 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
Clique table members: 12.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 4 threads.
Root relaxation solution time = 0.02 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

      0     0        3.0000    12                      3.0000       1

CASO 2: 
15 Jobs;
15 Machines;
225 Operations (15 Operation for Job);
T = 2000 (upper bound of Cmax or makespan)

In [4]:
using JuMP
using CPLEX

# Ejemplo de uso
J = 1:15  # conjunto de tareas
M = 1:15  # conjunto de maquinas
O = 1:225  # conjunto de operaciones

# tiempo de procesamiento para cada operacion
time = [94 66 10 53 26 15 65 82 10 27 93 92 96 70 83 
 74 31 88 51 57 78  8  7 91 79 18 51 18 99 33
  4 82 40 86 50 54 21  6 54 68 82 20 39 35 68
 73 23 30 30 53 94 58 93 32 91 30 56 27 92  9
 78 23 21 60 36 29 95 99 79 76 93 42 52 42 96
 29 61 88 70 16 31 65 83 78 26 50 87 62 14 30
 18 75 20  4 91 68 19 54 85 73 43 24 37 87 66
 32 52  9 49 61 35 99 62  6 62  7 80  3 57  7
 85 30 96 91 13 87 82 83 78 56 85  8 66 88 15
  5 59 30 60 41 17 66 89 78 88 69 45 82  6 13
 90 27  1  8 91 80 89 49 32 28 90 93  6 35 73
 47 43 75  8 51  3 84 34 28 60 69 45 67 58 87
 65 62 97 20 31 33 33 77 50 80 48 90 75 96 44
 28 21 51 75 17 89 59 56 63 18 17 30 16  7 35
 57 16 42 34 37 26 68 73  5  8 12 87 83 20 97]

# Maquina asignada para cada operacion
Machine = [7 13  5  8  4  3 11 12  9 15 10 14  6  1  2 
  5  6  8 15 14  9 12 10  7 11  1  4 13  2  3
  2  9 10 13  7 12 14  6  1  3  8 11  5  4 15
  6  3 10  7 11  1 14  5  8 15 12  9 13  2  4
  8  9  7 11  5 10  3 15 13  6  2 14 12  1  4
  6  4 13 14 12  5 15  8  3  2 11  1 10  7  9
 13  4  8  9 15  7  2 12  5  6  3 11  1 14 10
 12  6  1  8 13 14 15  2  3  9  5  4 10  7 11
 11 12  7 15  1  2  3  6 13  5  9  8 10 14  4
  7 12 10  3  9  1 14  4 11  8  2 13 15  5  6
  5  8 14  1  6 13  7  9 15 11  4  2 12 10  3
  3 15  1 13  7 11  8  6  9 10 14  2  4 12  5
  6  9 11  3  4  7 10  1 14  5  2 12 13  8 15
  9 15  5 14  6  7 10  2 13  8 12 11  4  3  1
 11  9 13  7  5  2 14 15 12  1  8  4  3 10  6]

T = 2000   # Upper bound para makespan
 
# calcular el numero de operaciones en la j-esima tarea
N_j=calculate_N_J(O, J)

# Transformamos las matrices Machine y time a vectores para ser utilizadas
M_i, p_i = matrix_to_vector(O, J, Machine, time)

makespan, start_times = solve_job_shop_scheduling(J, M, O, M_i, p_i, T, N_j)

println("Makespan: ", makespan)
println("Start times: ", start_times)

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
Tried aggregator 1 time.
MIP Presolve eliminated 225 rows and 47475 columns.
Reduced MIP has 6511 rows, 3376 columns, and 19322 nonzeros.
Reduced MIP has 3150 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (17.27 ticks)
Probing time = 0.00 sec. (1.46 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 6511 rows, 3376 columns, and 19322 nonzeros.
Reduced MIP has 3150 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (8.65 ticks)
Probing time = 0.00 sec. (2.04 ticks)
Clique table members: 2700.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 4 threads.
Root relaxation solution time = 0.11 sec. (102.40 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

      0     0      963.0000   408        

CASO 3: Variando el tiempo y las maquinas donde se procesan las operaciones
15 Jobs
15 Machines
225 Operations (15 Operation for Job)
T = 11000 (upper bound of Cmax or makespan)

In [2]:
using JuMP
using CPLEX

# Ejemplo de uso
J = 1:15  # conjunto de tareas
M = 1:15  # conjunto de maquinas
O = 1:225  # conjunto de operaciones

# tiempo de procesamiento para cada operacion
time = [86 60 10 59 65 94 71 25 98 49 43  8 90 21 73
 68 28 38 36 93 35 37 28 62 86 65 11 20 82 23
 33 67 96 91 83 81 60 88 20 62 22 79 38 40 82
 13 14 73 88 24 16 78 70 53 68 73 90 58  7  4
 93 52 63 13 19 41 71 59 19 60 85 99 73 95 19
 62 60 93 16 10 72 88 69 58 41 46 63 76 83 62
 50 68 90 34 44  5  8 25 70 53 78 92 62 85 70
 60 64 92 44 63 91 21  1 96 19 59 12 41 11 94
 93 46 51 37 91 90 63 40 68 13 16 83 49 24 23
  5 35 21 14 66  3  6 98 63 64 76 94 17 62 37
 35 42 62 68 73 27 52 39 41 25  9 34 50 41 98
 23 32 35 10 29 68 20  8 58 62 39 32  8 33 91
 28 31  3 28 66 59 24 45 81  8 44 42  2 23 53
 11 93 27 59 62 23 23  7 77 64 60 97 36 53 72
 36 98 38 24 84 47 72  1 91 85 68 42 20 30 30]

# Maquina asignada para cada operacion
Machine = [ 10 15  5 14 11  4  8  9  1  6  2  3 13  7 12
 11  9 12 15  4 14 10  8  5  3  7  2  6 13  1
  8  1  7  6 15 14  3 12  5 13  2 10  4 11  9
 10 12 15  1  2  9  6 11 13  5 14  4  7  8  3
 12  5 14  4  9  2 11 13  3 15  7  8  1 10  6
  6  3  2 11  1  5  9 15  7  4 10  8 12 13 14
  6 11 14  1 10  9  2 12 15  8 13  3  7  5  4
 13  1 10  4 14  7  6  8  3 15 12  9 11  2  5
 12 11  6 14  2 10  9  8  4  7  1  3 15 13  5
  3 15  4 11  7  2  1 14 12  5  6  9  8 13 10
 12 15 14  6  5 10  2  7 13  1  3  9 11  4  8
 13  4 11  9  5  8 14 12 15  2  3  1  6  7 10
  9 14  6  1 12 10  5 13  2 11  7  3  8 15  4
  3  6  5  4 10  2 12 14  8  7 11 15  1  9 13
  2 11  5  3  1  8  7 10 12 13  6 15  4 14  9]

T = 2000   # Upper bound para makespan
 
# calcular el numero de operaciones en la j-esima tarea
N_j=calculate_N_J(O, J)

# Transformamos las matrices Machine y time a vectores para ser utilizadas
M_i, p_i = matrix_to_vector(O, J, Machine, time)

makespan, start_times = solve_job_shop_scheduling(J, M, O, M_i, p_i, T, N_j)

println("Makespan: ", makespan)
println("Start times: ", start_times)

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
Tried aggregator 1 time.
MIP Presolve eliminated 225 rows and 47475 columns.
Reduced MIP has 6511 rows, 3376 columns, and 19322 nonzeros.
Reduced MIP has 3150 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.03 sec. (17.27 ticks)
Probing time = 0.00 sec. (1.46 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 6511 rows, 3376 columns, and 19322 nonzeros.
Reduced MIP has 3150 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (8.68 ticks)
Probing time = 0.00 sec. (2.05 ticks)
Clique table members: 2700.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 4 threads.
Root relaxation solution time = 0.12 sec. (102.88 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

      0     0      942.0000   404        

CASO 4: 20 jobs, 15 machines, 300 operations (15 operations for task)

In [ ]:
using JuMP
using CPLEX

# Ejemplo de uso
J = 1:20  # conjunto de tareas
M = 1:15  # conjunto de maquinas
O = 1:300  # conjunto de operaciones

# tiempo de procesamiento para cada operacion
time = [ 25 75 75 76 38 62 38 59 14 13 46 31 57 92  3
 67  5 11 11 40 34 77 42 35 96 22 55 21 29 16
 22 98  8 35 59 31 13 46 52 22 18 19 64 29 70
 99 42  2 35 11 92 88 97 21 56 17 43 27 19 23
 50  5 59 71 47 39 82 35 12  2 39 42 52 65 35
 48 57  5  2 60 64 86  3 51 26 34 39 45 63 54
 40 43 50 71 46 99 67 34  6 95 67 54 29 30 60
 59  3 85  6 46 49  5 82 18 71 48 79 62 65 76
 65 55 81 15 32 52 97 69 82 89 69 87 22 71 63
 70 74 52 94 14 81 24 14 32 39 67 59 18 77 50
 18  6 96 53 35 99 39 18 14 90 64 81 89 48 80
 44 75 12 13 74 59 71 75 30 93 26 30 84 91 93
 39 56 13 29 55 69 26  7 55 48 22 46 50 96 17
 57 14  8 13 95 53 78 24 92 90 68 87 43 75 94
 93 92 18 28 27 40 56 83 51 15 97 48 53 78 39
 47 34 42 28 11 11 30 14 10  4 20 92 19 59 28
 69 82 64 40 27 82 27 43 56 17 18 20 98 43 68
 84 26 87 61 95 23 88 89 49 84 12 51  3 44 20
 43 54 18 72 70 28 20 22 59 36 85 13 73 29 45
  7 97  4 22 74 45 62 95 66 14 40 23 79 34  8]

# Maquina asignada para cada operacion
Machine = [4 12 15  2 11  3  5  8  1 13  6 10  7 14  9
  6  1  4  9  5  2 13 15  7  8 11  3 10 14 12
  3  4 15  1 10 13  6  5  8 11  9 12 14  2  7
  9 11  2 14  4  5 15 10  3  6 12  8  1  7 13
 15  9  2  3 11 10 13  5  7  6  1 14  4 12  8
  4 11  2  6  7  1  9  8 12 14  3 15 13 10  5
  3 11  2 13  9  1  8  7 15 14  5  4  6 10 12
  2  1  3  5  8 14 12  4 13  6  7 15 10  9 11
  5  6 10 11  8  7  3  2 13  4 14  1  9 15 12
  2  5  4 11 15  1  7 14 12  9  6 13  8 10  3
  4 11  2  1 10  9 15  7  5  8  3 13  6 12 14
  3  8  7  9  4  6 15  5  2  1 10 11 14 12 13
  1  8 15  9 13 11 10  4  7  2  5  3 12 14  6
 13  4 10  5  2  1 11  7  6  3 15 14  8  9 12
  4 15  7  6 14 10  2  1 13  8  3  5 11  9 12
  6 15  7 13  9  3  5 10 12 14  4  2  8  1 11
  4  8 11 15  1  9  2 12  6 14  5 13  7 10  3
 11  9  3 12 14  7 15  4 10  8  5  6 13  1  2
  4  3 13 14  2  7 15  6  5  9 10 12  1 11  8
 12 15  6  7 11 10 14  2  5  9  1  4 13  3  8]

T = 1376   # Upper bound para makespan
 
# calcular el numero de operaciones en la j-esima tarea
N_j=calculate_N_J(O, J)

# Transformamos las matrices Machine y time a vectores para ser utilizadas
M_i, p_i = matrix_to_vector(O, J, Machine, time)

makespan, start_times = solve_job_shop_scheduling(J, M, O, M_i, p_i, T, N_j)

println("Makespan: ", makespan)
println("Start times: ", start_times)